<a href="https://colab.research.google.com/github/aloml2543/KB_OCR/blob/main/src/model1_!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
https://github.com/qjadud1994/CRNN-Keras
"""

'\nhttps://github.com/qjadud1994/CRNN-Keras\n'

In [ ]:
import tensorflow as tf
import matplotlib
import numpy as np
import os, random
import zipfile
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import pickle
import datetime

from keras import backend as K

from tensorflow import keras
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Input,GlobalMaxPooling2D,Dense, Conv2D, BatchNormalization, Activation, MaxPooling2D, Reshape, LSTM, Lambda, add, concatenate
from tensorflow.keras.models import Model, save_model, load_model
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

import numpy as np
import multiprocessing
from cv2 import resize
from os import path, listdir
from google.colab.patches import cv2_imshow
import cv2
import glob
plt = matplotlib.pyplot
img = matplotlib.image

plt.rcParams["figure.figsize"] = (25,1)

!python --version
print("tensorflow",tf.__version__)
print("numpy",np.__version__)
print("matplotlib",matplotlib.__version__)

Python 3.7.11
tensorflow 2.6.0
numpy 1.19.5
matplotlib 3.2.2


In [ ]:
#데이터셋 가져오기
!unzip "/content/drive/MyDrive/KB_OCR/DATA/snukb_dataset.zip" -d -n "/content"

Archive:  /content/drive/MyDrive/KB_OCR/DATA/snukb_dataset.zip
replace /content/test/images/30060.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Path to the data directory
data_dir = Path("/content/train/images/")
 
# Get list of all the images
train_label = pd.read_csv('/content/train/train.csv')
train_label.sort_values(by=['index'], axis=0, inplace=True, ascending=True)
image_dir = [str(index) + '.jpg' for index in list(train_label.index.to_numpy())]
labels = train_label.label.to_numpy()
characters = list(set(char for label in labels for char in label))


 
# Batch size for training and validation
batch_size = 1
 
# Desired image dimensions
img_width = 128
img_height = 64
 
max_length = max([len(label) for label in labels])
num_classes = len(characters) + 1

In [ ]:
# # Input data generator
def labels_to_text(labels):     # letters의 index -> text (string)
    return ''.join(list(map(lambda x: characters[int(x)], labels)))

def text_to_labels(text):
    output = list(map(lambda x: characters.index(x), text))
    while(len(output) != 38):
      output.append(1)    # text를 letters 배열에서의 인덱스 값으로 변환
    return output

In [ ]:
from tensorflow.keras.optimizers import Adam
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

def get_Model(training=True):
    input_shape = (img_width, img_height, 1)     # (128, 64, 1)

    # Make Networkw
    inputs = Input(name='the_input', shape=input_shape, dtype='float32')  # (None, 128, 64, 1)

    # Convolution layer (VGG)
    inner = Conv2D(64, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal')(inputs)  # (None, 128, 64, 64)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(2, 2), name='max1')(inner)  # (None,64, 32, 64)

    inner = Conv2D(128, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal')(inner)  # (None, 64, 32, 128)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(2, 2), name='max2')(inner)  # (None, 32, 16, 128)

    inner = Conv2D(256, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(inner)  # (None, 32, 16, 256)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = Conv2D(256, (3, 3), padding='same', name='conv4', kernel_initializer='he_normal')(inner)  # (None, 32, 16, 256)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(1, 2), name='max3')(inner)  # (None, 32, 8, 256)

    inner = Conv2D(512, (3, 3), padding='same', name='conv5', kernel_initializer='he_normal')(inner)  # (None, 32, 8, 512)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = Conv2D(512, (3, 3), padding='same', name='conv6')(inner)  # (None, 32, 8, 512)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(1, 2), name='max4')(inner)  # (None, 32, 4, 512)

    inner = Conv2D(512, (2, 2), padding='same', kernel_initializer='he_normal', name='con7')(inner)  # (None, 32, 4, 512)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)

    # CNN to RNN
    inner = Reshape(target_shape=((32, 2048)), name='reshape')(inner)  # (None, 32, 2048)
    inner = Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(inner)  # (None, 32, 64)

    # RNN layer
    lstm_1 = LSTM(256, return_sequences=True, kernel_initializer='he_normal', name='lstm1')(inner)  # (None, 32, 512)
    lstm_1b = LSTM(256, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='lstm1_b')(inner)
    reversed_lstm_1b = Lambda(lambda inputTensor: K.reverse(inputTensor, axes=1)) (lstm_1b)

    lstm1_merged = add([lstm_1, reversed_lstm_1b])  # (None, 32, 512)
    lstm1_merged = BatchNormalization()(lstm1_merged)
    
    lstm_2 = LSTM(256, return_sequences=True, kernel_initializer='he_normal', name='lstm2')(lstm1_merged)
    lstm_2b = LSTM(256, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='lstm2_b')(lstm1_merged)
    reversed_lstm_2b= Lambda(lambda inputTensor: K.reverse(inputTensor, axes=1)) (lstm_2b)

    lstm2_merged = concatenate([lstm_2, reversed_lstm_2b])  # (None, 32, 1024)
    lstm2_merged = BatchNormalization()(lstm2_merged)

    # transforms RNN output to character activations:
    inner = Dense(num_classes, kernel_initializer='he_normal',name='dense2')(lstm2_merged) #(None, 32, 63)
    y_pred = Activation('softmax', name='softmax')(inner)

    labels = Input(name='the_labels', shape=[max_length], dtype='float32') # (None ,8)
    input_length = Input(name='input_length', shape=[1], dtype='int64')     # (None, 1)
    label_length = Input(name='label_length', shape=[1], dtype='int64')     # (None, 1)

    # Keras doesn't currently support loss funcs with extra parameters
    # so CTC loss is implemented in a lambda layer
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length]) #(None, 1)

    if training:
        return Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)


# Get the model
model = get_Model()
model.summary()
ada = Adam()
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=ada)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          [(None, 128, 64, 1)] 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 64)  640         the_input[0][0]                  
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 64, 64)  256         conv1[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 64, 64)  0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
class TextImageGenerator:
    def __init__(self, img_dirpath,img_dir,label_df, img_w, img_h,
                 batch_size, downsample_factor, max_text_len=9):
        self.img_h = img_h
        self.img_w = img_w
        self.label_df = label_df
        self.batch_size = batch_size
        self.max_text_len = max_text_len
        self.downsample_factor = downsample_factor
        self.img_dirpath = img_dirpath                  # image dir path
        self.img_dir = img_dir     # images list
        self.n = len(self.img_dir)                      # number of images
        self.indexes = list(range(self.n))
        self.cur_index = 0
        self.imgs = np.zeros((self.n, self.img_h, self.img_w))
        self.texts = []

    ## samples의 이미지 목록들을 opencv로 읽어 저장하기, texts에는 label 저장
    def build_data(self):
        print(self.n, " Image Loading start...")
        for i, img_file in enumerate(self.img_dir):
            img = cv2.imread(self.img_dirpath + img_file, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (self.img_w, self.img_h))
            img = img.astype(np.float32)
            img = (img / 255.0) * 2.0 - 1.0
            label_df = self.label_df[self.label_df['index'] ==int(img_file[:-4])]

            self.imgs[i, :, :] = img
            self.texts.append(self.label_df.label.to_list()[0])
        print(len(self.texts) == self.n)
        print(self.n, " Image Loading finish...")

    def next_sample(self):      ## index max -> 0 으로 만들기
        self.cur_index += 1
        if self.cur_index >= self.n:
            self.cur_index = 0
            random.shuffle(self.indexes)
        return self.imgs[self.indexes[self.cur_index]], self.texts[self.indexes[self.cur_index]]

    def next_batch(self):       ## batch size만큼 가져오기
        while True:
            X_data = np.ones([self.batch_size, self.img_w, self.img_h, 1])     # (bs, 128, 64, 1)
            Y_data = np.ones([self.batch_size, self.max_text_len])             # (bs, 9)
            input_length = np.ones((self.batch_size, 1)) * (self.img_w // self.downsample_factor - 2)  # (bs, 1)
            label_length = np.zeros((self.batch_size, 1))           # (bs, 1)

            for i in range(self.batch_size):
                img, text = self.next_sample()
                img = img.T
                img = np.expand_dims(img, -1)
                X_data[i] = img
                Y_data[i] = text_to_labels(text)
                label_length[i] = len(text)

            # dict 형태로 복사
            inputs = {
                'the_input': X_data,  # (bs, 128, 64, 1)
                'the_labels': Y_data,  # (bs, 8)
                'input_length': input_length,  # (bs, 1) -> 모든 원소 value = 30
                'label_length': label_length  # (bs, 1) -> 모든 원소 value = 8
            }
            outputs = {'ctc': np.zeros([self.batch_size])}   # (bs, 1) -> 모든 원소 0
            yield (inputs, outputs)
num_cores = multiprocessing.cpu_count() # cpu core 개수

train_file_path = '/content/train/images/'
tiger_train = TextImageGenerator(train_file_path,tqdm(image_dir),train_label, 128, 64, batch_size, 4, max_text_len=max_length)
tiger_train.build_data()

  0%|          | 64/30060 [00:01<05:56, 84.08it/s] 

30060  Image Loading start...


100%|██████████| 30060/30060 [00:41<00:00, 728.73it/s]

True
30060  Image Loading finish...


In [ ]:
early_stopping_patience = 10

# Add early stopping
early_stopping = keras.callbacks.EarlyStopping(
    monitor="loss", patience=early_stopping_patience, restore_best_weights=True
)
now = datetime.datetime.now()
print("시작 시간:", now)
model.fit(tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    callbacks=[early_stopping],
                    epochs=500)

now = datetime.datetime.now()
print("종료 시간:", now)

Epoch 1/30
22884/30060 [=====================>........] - ETA: 28:11 - loss: 0.2071